# My overall objective is to create a congruency table (pearsons r correlation between every possible set of rows - which in my case are gene expressions). 

### The first cell contains the code that I found on stack exchange ( thanks piRSquared). The code works great for my application, but I now want to take the time to fully understand it and learn from it. 

http://stackoverflow.com/questions/40476713/congruency-table-in-pandas-pearson-correlation-between-each-row-for-every-row-p

### Functions within a function.... whoah dude 


In [96]:
import pandas as pd
import numpy as np
df = pd.read_csv("TPM_200_rows.csv") #importing sample data

In [ ]:
df_1_only_data = df.loc[:,"5GB1_FM40_T0m_TR2":"5GB1_FM40_T180m_TR1"] #taking only the rows with data in them 
df_2_transposed = df_1_only_data.T #transposing the data 

The following is the function within a function code that we want!! 

In [ ]:
import pandas as pd

def corr(df, step = len(df.index), mask_diagonal=False):

    n = df.shape[0]

    def corr_closure(df):
        d = df.values
        sums = d.sum(0, keepdims=True)
        stds = d.std(0, keepdims=True)

        def corr_(k=0, l=10):
            d2 = d.T.dot(d[:, k:l])
            sums2 = sums.T.dot(sums[:, k:l])
            stds2 = stds.T.dot(stds[:, k:l])

            return pd.DataFrame((d2 - sums2 / n) / stds2 / n,
                                df.columns, df.columns[k:l])

        return corr_

    c = corr_closure(df)

    step = min(step, df.shape[1])

    tups = zip(range(0, n, step), range(step, n + step, step))

    corr_table = pd.concat([c(*t) for t in tups], axis=1)

    if mask_diagonal:
        np.fill_diagonal(corr_table.values, np.nan)

    return corr_table

In [ ]:
corr(df_2_transposed) #it works if you try it :) 

In [ ]:
#But lets start with a simpler example to just understand how a function within a function could be useful
def foo_out0(var):
    var_out = var + ' - wee!'
    def foo_in_(abc):
        return var_out + abc
    return foo_in_


In [ ]:
foo_out0("poop")("pee") #OK what happened here? The first function return a function, to which you pass another argument

In [ ]:
#lets try this again but in a way that is easier to see
new_funct = foo_out0("poop")

In [ ]:
#again, passing the second argument to the second function 
new_funct("pee")

In [ ]:
#OK lets go back to the main function and try to understand it step by step

In [ ]:
def corr(df, step = len(df.index), mask_diagonal=False): #define function with input being df, and a kwarg of mask_diagonal 

    n = df.shape[0] #the shape along the 0 index (number of rows) 
    
    
    
        def corr_closure(df):  #this function will return another function (corr_)
            
            # https://docs.scipy.org/doc/numpy/glossary.html   (makes the following 3 lines easier to understand)
               
        d = df.values #a nd np array, list of lists. 
        
        sums = d.sum(0, keepdims=True) #sums of each column while collapsing axis = 0,
                                        #rows are collapsed, and dimension is retained 
            
        stds = d.std(0, keepdims=True)  #std of each column while collapsing axis = 0,
                                        #rows are collapsed, and dimension is retained

            def corr_(k=0, l=10): #defining a whole new function. why do we need 0 and 10?  
                
                d2 = d.T.dot(d[:, k:l])
                sums2 = sums.T.dot(sums[:, k:l])
                stds2 = stds.T.dot(stds[:, k:l])

                return pd.DataFrame((d2 - sums2 / n) / stds2 / n,
                                df.columns, df.columns[k:l])

        return corr_
    
    c = corr_closure(df)

    step = min(step, df.shape[1])

    tups = zip(range(0, n, step), range(step, n + step, step))

    corr_table = pd.concat([c(*t) for t in tups], axis=1)

    if mask_diagonal:
        np.fill_diagonal(corr_table.values, np.nan)

    return corr_table
    

### Whats up with df.values?

In [ ]:
#df.values creates a np n-d array. I believe the data was converted to a nd array in order to take sums 
#along row and maintain the dimensions (keep dims).

type(df_2_transposed.values)

In [ ]:
#lets try to take sums along both axis. if you sum along axis 1, that means you collapse axis-1 and return the sum of 
#each rows. 

d = df_2_transposed.values
d.sum(1, keepdims = True) #only 2 rows, these are their sums across columns

In [ ]:
#if you collapse along axis 0, this means that taking sums for every column. 

d.sum(0, keepdims= True) #should be over 4k columns with the sums of each one! 

### The dot product - diving into the second function. 

lets do the products of two 1d dataframes. 

In [84]:
A = pd.DataFrame({"a":[4,2,3,5]})
                  
B = pd.DataFrame({"a":[1,3,3,4]})


If we do A dot B, we would expect, 4*1 + 2*3 + 3*3 + 5*4 = 4 + 6 + 9 + 20 = 39

Interestingly, in a dataframe, I cannot do a dot product between the same length dataframe, but I can after I transpose it

In [120]:
A.T.dot(B)

,a
a,39


In [98]:
Q = np.array([4,2,3,5])
W = np.array([1,3,3,4])

In [101]:
Q.dot(W)

39

The numpy array works. Lets try a 2 day numpy array. 

In [135]:
Q1 = np.array([[4,2],[3,5],[2,1],[3,1]])  #each list is a new row, NOT a new column. 

In [136]:
W1 = np.array([[1,3,3,4],[3,2,1,1]])  #the second list 

In [137]:
Q1.dot(W1)

array([[10, 16, 14, 18],
       [18, 19, 14, 17],
       [ 5,  8,  7,  9],
       [ 6, 11, 10, 13]])

In [138]:
W1.dot(Q1)

array([[31, 24],
       [23, 18]])

So the dot product works as it should. be careful to have the right dimensions of vectors before taking their dot product. When multiplying (m x n) dot (n x p), the two ns must be the same length, and the resulting matrix is n x p.  

In [139]:
W1


array([[1, 3, 3, 4],
       [3, 2, 1, 1]])

In [141]:
W1[:,1:3]

array([[3, 3],
       [2, 1]])

### OK whats up with this line?
d2 = d.T.dot(d[:, k:l])

In [155]:
#Just a regular nd array 
W1

array([[1, 3, 3, 4],
       [3, 2, 1, 1]])

In [156]:
#just a regular nd array taken all along 0 axis (all rows), and 0:2 along 1 axis (columns)
W1[:,0:2] 

array([[1, 3],
       [3, 2]])

In [157]:
#transpose this baby 
W1.T

array([[1, 3],
       [3, 2],
       [3, 1],
       [4, 1]])

In [159]:
#to put it all together: This is transposed dataframe dot product with the same dataframe but for only a range of columns
W1.T.dot(W1[:,0:2] ) 

array([[10,  9],
       [ 9, 13],
       [ 6, 11],
       [ 7, 14]])

In [166]:
#this will result in a nd array that has 10 columns and same number of rows as the dataframe. 
D2 = df_2_transposed.values.T.dot(df_2_transposed.values[:,0:10])
D2.shape

(201, 10)

### I understand what d2 is but I dont undestand the purpose of doing it, sums are the same - Lets move to for now. 